In [30]:
from dotenv import load_dotenv

import pandas as pd
import os
import requests
import gspread
import numpy as np
from datetime import datetime
from google.oauth2.service_account import Credentials


load_dotenv(dotenv_path=r"D:\Python\Perso\Esport\venv\.env")

api_key = os.environ.get("api_riot_key")

In [45]:
def get_puuid(gameName:str, tagLine:str):
    root = "https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/"
    endpoint = f"{gameName}/{tagLine}"
    req = requests.get(root + endpoint + "?api_key=" + api_key).json()["puuid"]
    return req

def get_match_history(gameName:str, tagLine:str, start: int, type:str):
    puuid = get_puuid(gameName=gameName, tagLine = tagLine)
    root_url = "https://europe.api.riotgames.com"
    endpoint = f"/lol/match/v5/matches/by-puuid/{puuid}/ids"
    query_params = f"?type={type}&?start={start}&count=50"
    req = requests.get(root_url + endpoint + query_params + '&api_key=' + api_key).json()
    return req

def get_match_data_from_Id(matchId=None):
    root_url = "https://europe.api.riotgames.com"
    endpoint = f"/lol/match/v5/matches/{matchId}"
    req = requests.get(root_url + endpoint + '?api_key=' + api_key).json()
    return req 

# Trouver les index des joueurs :
def find_index_match_player(match,player_puuid_searched:str):
    match_puuid_participants = get_match_data_from_Id(match)["metadata"]["participants"]
    index_player = match_puuid_participants.index(player_puuid_searched)
    return int(index_player)

def find_index_opponent(player_index:int):
    if player_index <= 5:
        return player_index + 5
    else:
        return player_index - 5

# Toutes les autres que j'utilise souvent :
def get_match_Id(gameName: str, tagLine: str, champName: str = None, start_game: int = 0):
    """
    Récupère les ID des matchs basés sur les critères donnés.
    """
    puuid = get_puuid(gameName=gameName, tagLine=tagLine)
    list_matchs_ids = get_match_history(gameName=gameName, tagLine=tagLine, start=start_game, type="ranked")

    game_with_right_champ = []
    game_with_right_number = []
    
    nb_game_in_list = 0
    duration_minimum = 15

    # Parcourir tous les matchs récupérés
    for match_id in list_matchs_ids:
        game = get_match_data_from_Id(matchId=match_id)
        index_player = find_index_match_player(match=game, player_puuid_searched=puuid)
        
        if index_player == None:
            continue

        game_duration = round(game["info"]["gameDuration"] / 60)
        championName = game["info"]["participants"][index_player]["championName"]

        # Vérifier la durée du match
        if game_duration >= duration_minimum:
            nb_game_in_list += 1

            # Ajouter le match à la bonne liste (avec ou sans filtre sur le champion)
            if champName is None:
                game_with_right_number.append(match_id)
            elif championName == champName:
                game_with_right_champ.append(match_id)

    # Si on filtre par champion, on renvoie les matchs avec ce champion
    if champName:
        return game_with_right_champ
    else:
        return game_with_right_number
    
def info_par_role(gameName:str, tagLine:str, start_game:int, index_role:list[int,int] ,champName:str = None):
    """ Donne des informations en fonction des rôles.
     
    Args:
        gameName:str = Le nom du joueur
        tagLine:str = l'hashtag du joueur
        start_game:int 
        index_role:list[int,int] = Doit être donné par pair afin d'avoir uniquement les games jouer à ce rôle 
        champName:str = Doit donner le nom d'un champion
     """

    puuid = get_puuid(gameName = gameName, tagLine = tagLine)

    match_history = get_match_Id(gameName=gameName, tagLine=tagLine, start_game = start_game, champName = champName)
    
    df = pd.DataFrame()

    nb_game = 0

    for match_id in match_history:
        game = get_match_data_from_Id(match_id)
        index_player = find_index_match_player(match=game, player_puuid_searched=puuid)
        print(f"index_player : {index_player}")
        
        if index_player not in index_role:
            pass
        else:
            
            if index_player == None:
                pass

            try:
                info = game["info"]
            except:
                continue

            print(f"index_role : {index_role}")

            print(f"index player après if : {index_player}")

            participants = info["participants"]
            gameDuration = round(info["gameDuration"]/60)
            nb_game += 1

            #Nom du joueur : 
            riotIdGameName = participants[index_player]["riotIdGameName"]

            #Nom du champion : 
            champName = info["participants"][index_player]["championName"]

            # WIN/LOSE :
            win = participants[index_player]["win"]

            # Date de la game -> A REVOIR PAS SUR 
            gameStartTimestamp = datetime.datetime.fromtimestamp(info["gameStartTimestamp"]/1000).strftime("%d/%m/%Y")
            # KDA :
            kills = participants[index_player]["kills"]
            deaths = participants[index_player]["deaths"]
            assists = participants[index_player]["assists"]
            

            # VisionScore
            visionscore = participants[index_player]["visionScore"]

            # totalsbires + les quelques jgl
            total_sbires = participants[index_player]["totalMinionsKilled"] + participants[index_player]["neutralMinionsKilled"]

            #TotalDMG
            totalDamageDealtToChampions = participants[index_player]["totalDamageDealtToChampions"]

            # Gold% :
            goldEarned = participants[index_player]["goldEarned"]
            
            # XP :
            champExperience = participants[index_player]["champExperience"]

            # Jungle sbire :
            neutralMinionsKilled = participants[index_player]["neutralMinionsKilled"]

            #Total ping dans la game :
            assistMePings = participants[index_player]["assistMePings"]
            commandPings = participants[index_player]["commandPings"]
            enemyMissingPings = participants[index_player]["enemyMissingPings"]
            enemyVisionPings = participants[index_player]["enemyVisionPings"]
            holdPings = participants[index_player]["holdPings"]
            getBackPings = participants[index_player]["getBackPings"]
            needVisionPings = participants[index_player]["needVisionPings"]
            onMyWayPings = participants[index_player]["onMyWayPings"]
            pushPings = participants[index_player]["pushPings"]
            visionClearedPings = participants[index_player]["visionClearedPings"]
            total_ping = assistMePings + commandPings + enemyMissingPings + enemyVisionPings + holdPings + getBackPings + needVisionPings + onMyWayPings + pushPings + visionClearedPings

            # Ward : 
            total_ward_killed = participants[index_player]["challenges"]["wardTakedowns"] + participants[index_player]["wardsKilled"]
            wardsPlaced = participants[index_player]["wardsPlaced"] # Toutes les wards sont comptées
            nb_ward_bought = participants[index_player]["sightWardsBoughtInGame"] + participants[index_player]["visionWardsBoughtInGame"]
            
            controlWardsPlaced = participants[index_player]["challenges"]["controlWardsPlaced"]

            # Total kill du joueur : 
            if index_player in range(0, 5):
                team_indices = range(0, 5)  # Blue side
            else:
                team_indices = range(5, 10)  # Red side

            team_total_kills = sum(participants[i]["kills"] for i in team_indices)
            team_total_gold = sum(participants[i]["goldEarned"] for i in team_indices)
            gold_percentage = f"{round((goldEarned/team_total_gold) * 100)}%"

            player_kp_percentage = round((kills + assists) / team_total_kills * 100 if team_total_kills > 0 else 0)
            
            # J'essaie d'avoir le nombre 
            player_kp_with_percentage = f"{player_kp_percentage}%"

            # Pour le top laner
            if index_player == 0 or index_player == 5:
                top_df = pd.DataFrame({"Nom du joueur" : [riotIdGameName],
                                        "Date de la game" : [gameStartTimestamp],
                                        "Durée de la game" : [gameDuration],
                                        "champName" : [champName],
                                        "kills" : [kills],
                                        "deaths" : [deaths],
                                        "assists" : [assists],
                                        "total sbires tués" : [total_sbires],
                                        "Total DMG" : [totalDamageDealtToChampions],
                                        "Total Gold" : [goldEarned],
                                        "Gold%" : [gold_percentage],
                                        "Total XP" : [champExperience],
                                        "Vision Score" : [visionscore],
                                        "total_ping" : [total_ping],
                                        "%KP" : [player_kp_with_percentage],
                                        "win" : [win],
                })
                df = pd.concat([df, top_df], ignore_index=True)
            
            # Pour le jungler :
            elif index_player == 1 or index_player == 6:
                jungle_df = pd.DataFrame({"Nom du joueur" : [riotIdGameName],
                                        "Date de la game" : [gameStartTimestamp],
                                        "Durée de la game" : [gameDuration],
                                        "champName" : [champName],
                                        "kills" : [kills],
                                        "deaths" : [deaths],
                                        "assists" : [assists],
                                        "nb_ward_achetée" : [nb_ward_bought],
                                        "wardsPlaced" : [wardsPlaced], # Ward normal
                                        "Ward killed" : [total_ward_killed],
                                        "Monstres tués" : [neutralMinionsKilled],
                                        "Total DMG" : [totalDamageDealtToChampions],
                                        "Total Gold" : [goldEarned],
                                        "Gold%" : [gold_percentage],
                                        "Total XP" : [champExperience],
                                        "Vision Score" : [visionscore],
                                        "total_ping" : [total_ping],
                                        "%KP" : [player_kp_with_percentage],
                                        "win" : [win],

                })
                df = pd.concat([df, jungle_df],ignore_index=True)

            # Pour le mid :
            elif index_player == 2 or index_player == 7:
                mid_df = pd.DataFrame({"Nom du joueur" : [riotIdGameName],
                                        "Date de la game" : [gameStartTimestamp],
                                        "Durée de la game" : [gameDuration],
                                        "champName" : [champName],
                                        "kills" : [kills],
                                        "deaths" : [deaths],
                                        "assists" : [assists],
                                        "total sbires tués" : [total_sbires],
                                        "Total DMG" : [totalDamageDealtToChampions],
                                        "Total Gold" : [goldEarned],
                                        "Gold%" : [gold_percentage],
                                        "Total XP" : [champExperience],
                                        "Vision Score" : [visionscore],
                                        "total_ping" : [total_ping],
                                        "%KP" : [player_kp_with_percentage],
                                        "win" : [win],
                })
                df = pd.concat([df, mid_df],ignore_index=True)
            
            # Pour l'ADC :
            elif index_player == 3 or index_player == 8:
                adc_df = pd.DataFrame({"Nom du joueur" : [riotIdGameName],
                                        "Date de la game" : [gameStartTimestamp],
                                        "Durée de la game" : [gameDuration],
                                        "champName" : [champName],
                                        "kills" : [kills],
                                        "deaths" : [deaths],
                                        "assists" : [assists],
                                        "total sbires tués" : [total_sbires],
                                        "Total DMG" : [totalDamageDealtToChampions],
                                        "Total Gold" : [goldEarned],
                                        "Gold%" : [gold_percentage],
                                        "Total XP" : [champExperience],
                                        "Vision Score" : [visionscore],
                                        "total_ping" : [total_ping],
                                        "%KP" : [player_kp_with_percentage],
                                        "win" : [win],
                })
                df = pd.concat([df, adc_df],ignore_index=True)

            # Pour le support :
            elif index_player == 4 or index_player == 9:
                support_df = pd.DataFrame({"Nom du joueur" : [riotIdGameName],
                                        "Date de la game" : [gameStartTimestamp],
                                        "Durée de la game" : [gameDuration],
                                        "champName" : [champName],
                                        "kills" : [kills],
                                        "deaths" : [deaths],
                                        "assists" : [assists],
                                        "wardsPlaced" : [wardsPlaced], # Ward normal
                                        "Ward killed" : [total_ward_killed],
                                        "Pink ward placed" : [controlWardsPlaced],
                                        "Vision Score" : [visionscore],
                                        "Total XP" : [champExperience],
                                        "total_ping" : [total_ping],
                                        "%KP" : [player_kp_with_percentage],
                                        "win" : [win],
                })
                df = pd.concat([df, support_df],ignore_index=True)
    return df

# Pour la timeline
def get_timeline(match_id:str):
    root = "https://europe.api.riotgames.com"
    endpoint = f"/lol/match/v5/matches/{match_id}/timeline"
    return requests.get(root + endpoint + "?api_key=" + api_key).json()


In [46]:
get_match_Id(gameName="NEP Zerkome", tagLine = "EUW03")

SSLError: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v5/matches/%7B'metadata':%20%7B'dataVersion':%20'2',%20'matchId':%20'EUW1_7174593907',%20'participants':%20%5B'ZMrdNAUG-aDCDfhsRuLtrwAn3aI6CN-Y5DYGwrj1HjtbAZDDtPmIAl95cvF2rOYQxhNLXzvB_EagEw',%20'a8-GKY7RU-KDi18WQRfJTuuTRlK76906Af3njE2SUz_8GZjZ8od8_tx--pspLrGaYvmul45-wsXw3Q',%20'-pdsgEyN6z9Da1OdCyd5Xa8soMCy9nthGLBKLmGkVxZeofyydLopvixJJsA5tFVzz5YlM2VgBvkEsQ',%20'WXZRx2F8kBANuxbOoY3MosLu4eH0ZAG9nHOTyD9gfvEAhezotnCemmz_KFPkT86cRS3Xz8_ldA8spg',%20'3YG5VWdCQQ0NrVX917plMNLdW8AL2r98Z2qAVSwOHgKMGCVNPpcy8PcR1BU25d_t5zJVqHO9GiUpZQ',%20'0dSGrVosmQuIHs0-SW3so5x_BGJYqIZpI3jdeQ7Mb3NlmK29Umy-Ytzur3EaY-bLNA9KhwF8cCyQiQ',%20'ChxgO2YxAQt2H3N3IOKPLR07MVlyZg02ynVi1yrAXY5sbgBDp3O5sfu6DCLAbHjbrN-NGX7Lts-Q_g',%20'H7YA5cgXSTHo4xy6G-0vfv2cay6NzA0duqxIqzeHM2iA57gFuSE7wDBcQlph5AxCoco-1Eh19qoDfw',%20'2bVhBgZiSsbc_1WeYhq57lTxRASxzBmz1apo8KFL0j9s00wRTStN5l5XUH0fO--Xn16FnTDniPSJXA',%20'4rnwEWwQxdozSNY5m

In [28]:
timeline = get_timeline(match_id="EUW1_7170384119")
puuid_player = get_puuid(gameName="NEP Zerkome", tagLine = "EUW03")
index_player = find_index_match_player(match = "EUW1_7170384119", player_puuid_searched=puuid_player) + 1
index_opponent_tl = find_index_opponent(index_player)

timeline_frame = timeline["info"]["frames"]

timeline_frame[14]["participantFrames"][str(index_player)]["minionsKilled"] - timeline_frame[14]["participantFrames"][str(index_opponent_tl)]["minionsKilled"]

7

In [ ]:
matchs_Id = get_match_Id(gameName="NEP Zerkome", tagLine = "EUW03")
# puuid_player = get_puuid(gameName="NEP Zerkome", tagLine = "EUW03")
date_formatee = datetime.now().strftime("%d/%m/%Y")
game_today = 0
dict_metr = {}


matchs_Id
# for match_id in matchs_Id[0:20]:
#     game = get_match_data_from_Id(match_id)    
#     index_player = find_index_match_player(match = match_id, player_puuid_searched=puuid_player) + 1
#     index_opponent_tl = find_index_opponent(index_player)

#     gameStartTimestamp = datetime.fromtimestamp(game["info"]["gameStartTimestamp"]/1000).strftime("%d/%m/%Y")

#     if date_formatee == gameStartTimestamp:
#         game_today += 1
#         kda = game["info"]["participants"][index_player]["challenges"]["kda"]
#         vision_score = game["info"]["participants"][index_player]["visionScore"]
#         cs_d14 = timeline["info"]["frames"][15]["participantFrames"][str(index_player)]["minionsKilled"] - timeline["info"]["frames"][15]["participantFrames"][str(index_opponent_tl)]["minionsKilled"]


#         dict_metr = {"Nombre de game" : game_today,
#                      "csd@14" : np.median(cs_d14),
#                     "kda" : np.median(kda),
#                      "vision score" : np.median(vision_score)}
# dict_metr

SSLError: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v5/matches/%7B'metadata':%20%7B'dataVersion':%20'2',%20'matchId':%20'EUW1_7174593907',%20'participants':%20%5B'ZMrdNAUG-aDCDfhsRuLtrwAn3aI6CN-Y5DYGwrj1HjtbAZDDtPmIAl95cvF2rOYQxhNLXzvB_EagEw',%20'a8-GKY7RU-KDi18WQRfJTuuTRlK76906Af3njE2SUz_8GZjZ8od8_tx--pspLrGaYvmul45-wsXw3Q',%20'-pdsgEyN6z9Da1OdCyd5Xa8soMCy9nthGLBKLmGkVxZeofyydLopvixJJsA5tFVzz5YlM2VgBvkEsQ',%20'WXZRx2F8kBANuxbOoY3MosLu4eH0ZAG9nHOTyD9gfvEAhezotnCemmz_KFPkT86cRS3Xz8_ldA8spg',%20'3YG5VWdCQQ0NrVX917plMNLdW8AL2r98Z2qAVSwOHgKMGCVNPpcy8PcR1BU25d_t5zJVqHO9GiUpZQ',%20'0dSGrVosmQuIHs0-SW3so5x_BGJYqIZpI3jdeQ7Mb3NlmK29Umy-Ytzur3EaY-bLNA9KhwF8cCyQiQ',%20'ChxgO2YxAQt2H3N3IOKPLR07MVlyZg02ynVi1yrAXY5sbgBDp3O5sfu6DCLAbHjbrN-NGX7Lts-Q_g',%20'H7YA5cgXSTHo4xy6G-0vfv2cay6NzA0duqxIqzeHM2iA57gFuSE7wDBcQlph5AxCoco-1Eh19qoDfw',%20'2bVhBgZiSsbc_1WeYhq57lTxRASxzBmz1apo8KFL0j9s00wRTStN5l5XUH0fO--Xn16FnTDniPSJXA',%20'4rnwEWwQxdozSNY5m

# Conf google sheets api

In [2]:
scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]
cred = Credentials.from_service_account_file(r"D:\Python\Perso\Esport\JSONs\riot-api-test-439419-b8dcea6f326c.json",scopes=scopes)
client = gspread.authorize(cred)

sheets_id = "1hHYDu6wypqq36_us6GxsVwofLNaUsMzDq1bSs9t8zl8"
workbook = client.open_by_key(sheets_id)

## Faire joue-joue avec l'API google sheets

In [3]:
# Avoir tous les noms des sheets
sheets = map(lambda x: x.title, workbook.worksheets())
bark = print(list(sheets))
bark

['NEP Zerkome', 'jungle abuser', 'TIMESH', 'Copie de TIMESH', 'Please', 'OTPchromosome21', 'Ragnar Lothbrok']


# Codes

In [ ]:
class rank_info_gsheets():
    def __init__(self,gameName:str,tagLine:str) -> None:
        self.gameName = gameName
        self.tagLine = tagLine
        self.encryptedSummonerId = self.get_encryptedSummonerId()
    
    def get_encryptedSummonerId(self):
        puuid = get_puuid(gameName=self.gameName, tagLine=self.tagLine)
        return requests.get(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={api_key}").json()["id"]
    
    def get_rank_info(self):
        rank_info = {}
        encryptedSummonerId = self.encryptedSummonerId
        req = requests.get(f"https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/{encryptedSummonerId}?api_key={api_key}").json()

        if "RANKED_SOLO_5x5" not in req[0]["queueType"]:
            rank_info["queueType"] = req[1]["queueType"]
            rank_info["tier"] = req[1]["tier"]
            rank_info["rank"] = req[1]['rank']
            rank_info["lp"] = req[1]["leaguePoints"]
            rank_info["wins"] = req[1]["wins"]
            rank_info["losses"] = req[1]["losses"]
        else:
            rank_info["queueType"] = req[0]["queueType"]
            rank_info["tier"] = req[0]["tier"]
            rank_info["rank"] = req[0]['rank']
            rank_info["lp"] = req[0]["leaguePoints"]
            rank_info["wins"] = req[0]["wins"]
            rank_info["losses"] = req[0]["losses"]

        return rank_info
    
    def empty_row(row:str,next_ligne:int = 1):
        numbers = 0
        for letter in row:
            try:
                numbers = int(letter) + next_ligne
            except ValueError:
                last_letter = letter
        next_row = last_letter + str(numbers)
        return next_row
    
    def create_worksheet(self):
        username = get_user_info(gameName=self.gameName, tagLine=self.tagLine)
        workbook.add_worksheet(username, rows = 100, cols=100)

In [ ]:
rank = rank_info_gsheets(gameName="NEP Zerkome",tagLine="EUW03")
rank.get_rank_info()

{'queueType': 'RANKED_SOLO_5x5',
 'tier': 'MASTER',
 'rank': 'I',
 'lp': 308,
 'wins': 165,
 'losses': 152}

In [107]:
def get_encryptedSummonerId(gameName:str, tagLine:str):
    puuid = get_puuid(gameName=gameName, tagLine=tagLine)
    return requests.get(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={api_key}").json()["id"]

def get_rank_info(gameName:str, tagLine:str):
    rank_info = {}
    encryptedSummonerId = get_encryptedSummonerId(gameName=gameName, tagLine=tagLine)
    req = requests.get(f"https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/{encryptedSummonerId}?api_key={api_key}").json()

    if "RANKED_SOLO_5x5" not in req[0]["queueType"]:
        # rank_info["queueType"] = req[1]["queueType"]
        rank_info["tier"] = req[1]["tier"]
        rank_info["rank"] = req[1]['rank']
        rank_info["lp"] = req[1]["leaguePoints"]
        rank_info["wins"] = req[1]["wins"]
        rank_info["losses"] = req[1]["losses"]
        rank_info["Number game"] = int(req[1]["wins"]) + int(req[1]["losses"])
    else:
        # rank_info["queueType"] = req[0]["queueType"]
        rank_info["tier"] = req[0]["tier"]
        rank_info["rank"] = req[0]['rank']
        rank_info["lp"] = req[0]["leaguePoints"]
        rank_info["wins"] = req[0]["wins"]
        rank_info["losses"] = req[0]["losses"]
        rank_info["Number game"] = int(req[0]["wins"]) + int(req[0]["losses"])
    return rank_info

def empty_row(row:str,next_ligne:int = 1):
    numbers = 0
    for letter in row:
        try:
            numbers = int(letter) + next_ligne
        except ValueError:
            last_letter = letter
    next_row = last_letter + str(numbers)
    return next_row

def permutate_row_number(row:str, letter_to_permutate:list[str]):
    current_row = empty_row(row = row, next_ligne = 0)
    for letter in current_row:
        try:
            number = int(letter)
        except ValueError:
            pass
    
    all_row = []
    for single_letter in letter_to_permutate:
        all_row.append(single_letter + str(number))
    return all_row

def create_worksheet(gameName:str, tagLine:str):
    username = get_user_info(gameName=gameName, tagLine=tagLine)
    sheets_name = list(map(lambda x: x.title, workbook.worksheets()))
    
    if username not in sheets_name:
        player_sheet = workbook.add_worksheet(username, rows = 100, cols=100)
        player_sheet.update_acell("B1","Rank")
        player_sheet.update_acell("C1","Division")
        player_sheet.update_acell("D1","LP")
        player_sheet.update_acell("E1","Wins")
        player_sheet.update_acell("F1","Loses")
        player_sheet.update_acell("G1","Nombre de game")
        
        # Formatter le sheets automatiquement :
        player_sheet.format(["A1:G1"],format={
            "borders" : { 
                "top" : {"style" : "SOLID_MEDIUM"},
                "bottom" : {"style" : "SOLID_MEDIUM"},
                "left" : {"style" : "SOLID_MEDIUM"},
                "right" : {"style" : "SOLID_MEDIUM"}},
            "horizontalAlignment" : "CENTER",
            "verticalAlignment" : "MIDDLE",
            })
        
        player_sheet.format(["A2:A10"],format={
            "borders" : { 
                "top" : {"style" : "SOLID_MEDIUM"},
                "bottom" : {"style" : "SOLID_MEDIUM"},
                "left" : {"style" : "SOLID_MEDIUM"},
                "right" : {"style" : "SOLID_MEDIUM"}},
            "horizontalAlignment" : "CENTER",
            "verticalAlignment" : "MIDDLE",
            })

        player_sheet.format(["B2:G10"], format={
            "borders": {
                "top": {"style": "SOLID"},
                "bottom": {"style": "SOLID"},
                "left": {"style": "SOLID"},
                "right": {"style": "SOLID"}},
            "horizontalAlignment" : "CENTER",
            "verticalAlignment" : "MIDDLE",
            })

        return player_sheet
    else:
        player_sheet = workbook.worksheet(username)
        return player_sheet

In [ ]:
def update_gsheet(gameName:str, tagLine:str):
    # Avoir les informations des joueurs dans la ranked.
    rank_info = get_rank_info(gameName=gameName,tagLine=tagLine)
    # Selectionner la page de la personne que l'on veut scout
    player_sheet = create_worksheet(gameName=gameName,tagLine=tagLine)
    number = 0
    
    # Afficher la date
    date_formatee = datetime.now().strftime("%d/%m/%Y")

    # Avoir la prochaine cellule
    next_row = empty_row("A1", next_ligne = 1)

    # Avoir la valeure de la cellule que l'on veut
    next_row_value = player_sheet.acell(next_row).value
    if next_row_value == None:
        player_sheet.update_acell(next_row, date_formatee)
        column_row = permutate_row_number(row = next_row, letter_to_permutate = ["B","C","D","E","F","G","H"])
        for idx, (key, value) in enumerate(rank_info.items()):
            player_sheet.update_acell(column_row[idx], value)

    # Tant que la valeur de la cellule est différent de None
    # Alors tu fais parcours la cellule d'en dessous et tu rechecks
    while next_row_value != None:
        number += 1
        next_row_value = player_sheet.acell(empty_row("A1", next_ligne = number)).value
        next_row = empty_row("A1", next_ligne = number)

        if next_row_value == date_formatee:
            print("Vous l'avez déjà fait aujourd'hui !")
            question = input("Voulez-vous faire une mise à jour : [Y/N]")
            
            if question == "N":
                print("Fin du programme.")
                break

            else:
                print("Mise à jours en cours...")
                current_row_out = empty_row("A1", next_ligne = number)
                player_sheet.update_acell(current_row_out, date_formatee)
                column_row = permutate_row_number(row = current_row_out, letter_to_permutate = ["B","C","D","E","F","G","H"])
                for idx, (key, value) in enumerate(rank_info.items()):
                    player_sheet.update_acell(column_row[idx], value)
                print("Mise à jour terminée.")
                break
        
        if next_row_value == None:
            player_sheet.update_acell(next_row, date_formatee)
            column_row = permutate_row_number(row = next_row, letter_to_permutate = ["B","C","D","E","F","G","H"])
            for idx, (key, value) in enumerate(rank_info.items()):
                player_sheet.update_acell(column_row[idx], value)

In [ ]:
# Update le sheets automatiquement, à mettre sous forme de fonction :
def auto_update_gsheet():
    workbook.worksheet(username)
    scout_sheet = workbook.worksheet("Joueur à Scout")
    add_player = str(input("Voulez-vous ajoutez un joueur à scout ? [Y/N]"))
    index = 0
    if add_player == "Y":
        # gameName_player = str(input("Quel est son gameName : "))
        # tagLine_player = str(input("Quel est son tagLine : "))
        index = scout_sheet.update_acell(empty_row("A1",next_ligne = 1)).values
        print(index)
        
        
    # scout_sheet = workbook.worksheet("Joueur à Scout")
    # gameName_col = scout_sheet.col_values(col=3)
    # tagline_col = scout_sheet.col_values(col=4)
    # for x in range(1,len(tagline_col)):
    #     print(f"Le joueur en cours : {gameName_col[x]}")
    #     update_gsheet(gameName=gameName_col[x], tagLine=tagline_col[x])

In [13]:
auto_update_gsheet()

NameError: name 'workbook' is not defined

In [ ]:
# Avoir les rôles sur gsheet
# Si c'est un nouveau joueur ça met pas les informations